In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/Clinic/master/ultima_all_clean.csv')
df = df.set_index('hn')
df.head()

,age,parity,hiv,menopaus,disease,surgery,conization,OPDsize,appearance,stage,...,nodeyiel,RHlvsi,depth,size,utmet,vgmargin,vgmet,pelvicme,pmmet,adnmet
hn,,,,,,,,,,,,,,,,,,,,,
2631840,52,3,0.0,0.0,1,1,0.0,5.0,4.0,5.0,...,21.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2633481,32,2,0.0,0.0,0,1,0.0,5.0,1.0,5.0,...,11.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0
2634477,52,2,0.0,0.0,0,0,0.0,5.0,NaN,5.0,...,35.0,6.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2633633,38,2,0.0,0.0,0,0,0.0,2.0,1.0,4.0,...,20.0,16.0,3.0,3.8,0.0,0.0,0.0,0.0,0.0,2.0
2630496,55,3,0.0,1.0,0,0,1.0,0.0,5.0,4.0,...,17.0,9.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [3]:
df.shape

(1723, 23)

In [4]:
df.isnull().sum(axis=0)

age             0
parity          0
hiv             4
menopaus        1
disease         0
surgery         0
conization      5
OPDsize        17
appearance    101
stage          24
pchemo          1
Wardsize      145
finalhisto     10
nodeyiel       12
RHlvsi        366
depth         489
size          114
utmet          98
vgmargin       96
vgmet          97
pelvicme        1
pmmet          94
adnmet          7
dtype: int64

In [5]:
df_clean_column = df.drop(['appearance','Wardsize','RHlvsi','depth','nodeyiel','vgmargin','pelvicme','adnmet'],axis = 1)
df_clean_column.isnull().sum(axis=0)

age             0
parity          0
hiv             4
menopaus        1
disease         0
surgery         0
conization      5
OPDsize        17
stage          24
pchemo          1
finalhisto     10
size          114
utmet          98
vgmet          97
pmmet          94
dtype: int64

In [6]:
df_clean = df_clean_column.dropna(axis = 0)

In [7]:
df_clean.shape

(1555, 15)

In [8]:
pel_class = np.unique(df_clean['pmmet'])
pel_class

array([0., 1.])

In [9]:
pel_value = [sum(df_clean['pmmet']==pel_class[0]),
             sum(df_clean['pmmet']==pel_class[1])]
pel_value

[1278, 277]

In [10]:
y = df_clean['pmmet'].values
X = df_clean.drop(['pmmet'],axis = 1).values

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=0)

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)

In [14]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_val_norm)
acc = sum(yp == y_val)/len(y_val)
print("Hold Out Training Accuracy : "+str(acc*100))

Hold Out Training Accuracy : 83.53413654618474


# Cross Validation

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)

In [17]:
from sklearn.model_selection import cross_val_score
clf = LogisticRegression(random_state=0, solver='liblinear')
acc = cross_val_score(clf, X_train_norm, y_train, cv=10)
print("10CV Training Accuracy : " +str(acc.mean()*100))

10CV Training Accuracy : 83.36193548387098


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [20]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test Training Accuracy : "+str(acc*100))

Test Training Accuracy : 84.56591639871382


In [21]:
df.var(axis=0)

age               81.065940
parity             1.795348
hiv                0.006936
menopaus           0.207860
disease            0.215740
surgery            0.234638
conization         0.226518
OPDsize            3.664245
appearance         2.997285
stage              1.383333
pchemo             0.149920
Wardsize           3.361144
finalhisto         0.744274
nodeyiel         114.208070
RHlvsi        231725.557678
depth              0.509143
size               3.207926
utmet              0.153174
vgmargin           0.189784
vgmet              0.357326
pelvicme           0.388821
pmmet              0.146025
adnmet             0.732903
dtype: float64

In [22]:
df.var(axis=1)

hn
2631840      126.565217
2633481       46.723320
2634477      162.833333
2633633       79.753518
2630496      137.418972
               ...     
3169688      159.447470
3167041       82.158009
2623351      104.514286
3302539       97.457143
3855674    43128.259881
Length: 1723, dtype: float64

In [23]:
df.shape

(1723, 23)

In [24]:
df_nodup = df.drop_duplicates()
df_nodup.shape

(1723, 23)

In [25]:
s = df.corr().unstack()
so = s.sort_values(ascending=False)
so

age         age           1.000000
finalhisto  finalhisto    1.000000
menopaus    menopaus      1.000000
disease     disease       1.000000
surgery     surgery       1.000000
                            ...   
conization  Wardsize     -0.711525
appearance  Wardsize     -0.757519
Wardsize    appearance   -0.757519
appearance  OPDsize      -0.790494
OPDsize     appearance   -0.790494
Length: 529, dtype: float64

In [26]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
X_train_pca.shape

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_pca)
X_train_norm = scaler.transform(X_train_pca)
X_test_norm = scaler.transform(X_test_pca)

In [27]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test Training Accuracy : "+str(acc*100))

Test Training Accuracy : 85.53054662379421


In [28]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 3)
tsne.fit(X_train)
X_train_tsne = pca.fit_transform(X_train)
X_test_tsne = pca.fit_transform(X_test)
X_train_tsne.shape

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_tsne)
X_train_norm = scaler.transform(X_train_tsne)
X_test_norm = scaler.transform(X_test_tsne)

In [29]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test Training Accuracy : "+str(acc*100))

Test Training Accuracy : 84.88745980707395


# SelectKBest

In [30]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
fs = SelectKBest(f_classif, k=6)
fs.fit(X_train, y_train)
X_train_fs = fs.transform(X_train)
X_test_fs = fs.transform(X_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X_train_fs)
X_train_norm = scaler.transform(X_train_fs)
X_test_norm = scaler.transform(X_test_fs)

In [31]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Hold out training accuracy : "+str(acc*100))

Hold out training accuracy : 85.85209003215434


In [32]:
np.argsort(-fs.scores_)

array([11,  7,  8,  6, 12, 13,  3,  0, 10,  2,  4,  1,  5,  9])

# Pipeline

In [33]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)

params = {'C' : [1, 2, 4, 8, 16]}

from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))),
  ('feature_selection', SelectKBest(f_classif, k=5)),
  ('classification', GridSearchCV(LogisticRegression(random_state=0, solver='liblinear'),params, cv=5))
])
clf.fit(X_res, y_res)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest(k=5)),
                ('classification',
                 GridSearchCV(cv=5,
                              estimator=LogisticRegression(random_state=0,
                                                           solver='liblinear'),
                              param_grid={'C': [1, 2, 4, 8, 16]}))])

In [34]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(clf, X_res, y_res, cv=10)
print("10CV Training Accuracy : "+str(acc.mean()))

10CV Training Accuracy : 0.7509803921568627


In [35]:
yp = clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc*100))

Test accuracy : 71.06109324758843


# *Assignment*

In [36]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)

params = { 'n_estimators' : [10,50,100,200,500],
    'min_samples_leaf' : [1,2,4,8,16,32],
    'max_features' : ['sqrt',0.5,0.8],
    'criterion' : ['gini','entropy']
         }

from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))),
  ('feature_selection', SelectKBest(f_classif, k=10)),
  ('classification', GridSearchCV(RandomForestClassifier(random_state=0),params, cv = 10))
])
clf.fit(X_res, y_res)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest()),
                ('classification',
                 GridSearchCV(cv=10,
                              estimator=RandomForestClassifier(random_state=0),
                              param_grid={'criterion': ['gini', 'entropy'],
                                          'max_features': ['sqrt', 0.5, 0.8],
                                          'min_samples_leaf': [1, 2, 4, 8, 16,
                                                               32],
                                          'n_estimators': [10, 50, 100, 200,
                                                           500]}))])

In [37]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(clf, X_res, y_res, cv=10)
print("10CV Training Accuracy : "+str(acc.mean()*100))

10CV Training Accuracy : 88.62745098039217


In [38]:
yp = clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc*100))

Test accuracy : 81.67202572347267
